In [0]:
%pip install mlflow>3.1 openai
%restart_python

### Set MLflow Experiment to log traces from app

In [0]:
import mlflow

mlflow.set_experiment("/Users/alex.miller@databricks.com/multi-agent-supervisor")

### OpenAI client example

In [0]:
from openai import OpenAI
import mlflow
import os


mlflow.openai.autolog()
# Alternatively in a Databricks notebook you can use this:
DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

client = OpenAI(
    api_key=DATABRICKS_TOKEN,
    base_url="https://e2-demo-west.cloud.databricks.com/serving-endpoints"
)

response = client.responses.create(
    model="mas-3bfe8584-endpoint",
    input=[
        {
            "role": "user",
            "content": "What is month-over-month revenue growth by SKU?"
        }
    ]
)

print(response.output[0].content[0].text)

### Workspace client example

In [0]:
import os
from databricks.sdk import WorkspaceClient
# or: from databricks.sdk import WorkspaceClient; client = WorkspaceClient().serving_endpoints.get_open_ai_client()

client = WorkspaceClient().serving_endpoints.get_open_ai_client()

print("creating stream")
# Stream tokens from a chat model served by Databricks (example model name shown)
stream = client.responses.create(
    model="mas-3bfe8584-endpoint",
    input=[
        {"role": "user", "content": "What is the projected ending on hand and stockout risk?"}
    ],
    stream=True  # <- streaming enabled
)

response_text = ""
for chunk in stream:
    if chunk.type in ("response.output_text.delta", "response.output_item_done"):
      print(chunk)
#     response_text += chunk.content  # Or chunk['content'] if chunks are dicts
# print(response_text)

### ResponsesAgent Example (use this for production app deployment)

In [0]:
import os
from typing import Generator
from databricks.sdk import WorkspaceClient
import mlflow
from mlflow.entities.span import SpanType
from mlflow.pyfunc import ResponsesAgent
from mlflow.types.responses import (
    ResponsesAgentRequest,
    ResponsesAgentResponse,
    ResponsesAgentStreamEvent,
)

class SimpleResponsesAgent(ResponsesAgent):
    def __init__(self, model: str):
        self.client = WorkspaceClient().serving_endpoints.get_open_ai_client()
        self.model = model

    @mlflow.trace(span_type=SpanType.AGENT)
    def predict_stream(
        self, request: ResponsesAgentRequest
    ) -> Generator[ResponsesAgentStreamEvent, None, None]:
        for event in self.client.responses.create(
            input=request.input, stream=True, model=self.model
        ):
            yield ResponsesAgentStreamEvent(**event.to_dict())

    @mlflow.trace(span_type=SpanType.AGENT)
    def predict(
        self, request: ResponsesAgentRequest
    ) -> ResponsesAgentResponse:
        response = self.client.responses.create(
            input=request.input, stream=False, model=self.model
        )
        # Assume the response is compatible with ResponsesAgentResponse
        return ResponsesAgentResponse(**response.to_dict())


In [0]:
agent = SimpleResponsesAgent("mas-3bfe8584-endpoint")
request = ResponsesAgentRequest(
    input=[
        {"role": "user", "content": "What is the projected ending on hand and stockout risk?"}
    ]
)
response = agent.predict(request)
print(response)

In [0]:
for event in agent.predict_stream(request):
    print(event)

### Log user feedback from app example

In [0]:
import mlflow
from mlflow.entities.assessment import AssessmentSource, AssessmentSourceType

trace_id = mlflow.get_last_active_trace_id()
mlflow.log_feedback(
    trace_id=trace_id,
    name="user_feedback",
    value=True,  # or False, or a score
    rationale="User found the answer helpful",
    source=AssessmentSource(source_type=AssessmentSourceType.HUMAN, source_id="user_123"),
)
